# Assignment 4

# Table of contents
1. [Introduction](#introduction)

2. [Dataset Analysis](#question1)

3. [Apply models in Naïve Bayes, Logistic Regression, DecisionTree, Random Forest, Gradient Boosting and SVM](#question2)

4. [Apply PCA, SelectKBest and RFE for feature selections](#question3)

5. [Using gridsearchCV to obtain best parameters for modelsf](#question4)

6. [Compare performances among models, write up analysis why the model is good or bad in the algorithmic approach](#question5)

7. [Conclusions](#question6)

## 1. Introduction <a name="introduction"></a>
This project use dataset from http://stat-computing.org/dataexpo/2009/the-data.html to predict how late flights
will be. A flight only counts as late if it is more than 30 minutes late. The project will answer 6 following questions:
1. Dataset Analysis
2. Apply models in Naïve Bayes, Logistic Regression, DecisionTree, Random Forest, Gradient Boosting and SVM.
3. Apply PCA, SelectKBest and RFE for feature selections.
4. Using gridsearchCV to obtain best parameters for models.
5. Compare performances among models, write up analysis why the model is good or bad in the algorithmic approach (explain why the algorithm is good or bad for the dataset structure, can you do something to improve the model?)
6. Include the conclusions.

## 2. Dataset Analysis<a name="dataset"></a>

#### Import Python libraries<a name="library"></a>

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn 
import matplotlib.pyplot as plt
%matplotlib inline
import zipfile
import time
from imblearn.over_sampling import SMOTE

#### Import data<a name="data"></a>

In [2]:
url = "D:\Hoang\SauDaiHoc\Python\CBD Robotics\Assignment 5\delay.bz2"
data = pd.read_csv(url)

#### Data description<a name="description"></a>

In [3]:
# Number of rows and columns
print("Shape of dataframe is: {}".format(data.shape))

Shape of dataframe is: (7009728, 29)


In [4]:
# Dataset columns
data.columns

Index(['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime',
       'ArrTime', 'CRSArrTime', 'UniqueCarrier', 'FlightNum', 'TailNum',
       'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay',
       'DepDelay', 'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut',
       'Cancelled', 'CancellationCode', 'Diverted', 'CarrierDelay',
       'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay'],
      dtype='object')

In [5]:
# Data types
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7009728 entries, 0 to 7009727
Data columns (total 29 columns):
Year                 int64
Month                int64
DayofMonth           int64
DayOfWeek            int64
DepTime              float64
CRSDepTime           int64
ArrTime              float64
CRSArrTime           int64
UniqueCarrier        object
FlightNum            int64
TailNum              object
ActualElapsedTime    float64
CRSElapsedTime       float64
AirTime              float64
ArrDelay             float64
DepDelay             float64
Origin               object
Dest                 object
Distance             int64
TaxiIn               float64
TaxiOut              float64
Cancelled            int64
CancellationCode     object
Diverted             int64
CarrierDelay         float64
WeatherDelay         float64
NASDelay             float64
SecurityDelay        float64
LateAircraftDelay    float64
dtypes: float64(14), int64(10), object(5)
memory usage: 1.5+ GB


In [6]:
data.isnull().sum()

Year                       0
Month                      0
DayofMonth                 0
DayOfWeek                  0
DepTime               136246
CRSDepTime                 0
ArrTime               151649
CRSArrTime                 0
UniqueCarrier              0
FlightNum                  0
TailNum                83365
ActualElapsedTime     154699
CRSElapsedTime           844
AirTime               154699
ArrDelay              154699
DepDelay              136246
Origin                     0
Dest                       0
Distance                   0
TaxiIn                151649
TaxiOut               137058
Cancelled                  0
CancellationCode     6872294
Diverted                   0
CarrierDelay         5484993
WeatherDelay         5484993
NASDelay             5484993
SecurityDelay        5484993
LateAircraftDelay    5484993
dtype: int64

In [7]:
# Percentage of missing values
data.isnull().sum()/len(data)*100

Year                  0.000000
Month                 0.000000
DayofMonth            0.000000
DayOfWeek             0.000000
DepTime               1.943670
CRSDepTime            0.000000
ArrTime               2.163408
CRSArrTime            0.000000
UniqueCarrier         0.000000
FlightNum             0.000000
TailNum               1.189276
ActualElapsedTime     2.206919
CRSElapsedTime        0.012040
AirTime               2.206919
ArrDelay              2.206919
DepDelay              1.943670
Origin                0.000000
Dest                  0.000000
Distance              0.000000
TaxiIn                2.163408
TaxiOut               1.955254
Cancelled             0.000000
CancellationCode     98.039382
Diverted              0.000000
CarrierDelay         78.248300
WeatherDelay         78.248300
NASDelay             78.248300
SecurityDelay        78.248300
LateAircraftDelay    78.248300
dtype: float64

In [24]:
# To remove an entire column
remove = ['Year','DepTime','ArrTime','ActualElapsedTime','CRSElapsedTime','AirTime','TailNum','FlightNum',
          'UniqueCarrier','TaxiIn','TaxiOut','Cancelled','CancellationCode','Diverted','CarrierDelay',
          'WeatherDelay','NASDelay','SecurityDelay','LateAircraftDelay',
          'Origin','Dest']
data1= data.drop(remove, axis=1)
data1

,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,ArrDelay,DepDelay,Distance
0,1,3,4,1955,2225,-14.0,8.0,810
1,1,3,4,735,1000,2.0,19.0,810
2,1,3,4,620,750,14.0,8.0,515
3,1,3,4,930,1100,-6.0,-4.0,515
4,1,3,4,1755,1925,34.0,34.0,515
...,...,...,...,...,...,...,...,...
7009723,12,13,6,959,1150,14.0,3.0,533
7009724,12,13,6,835,1023,-2.0,-1.0,874
7009725,12,13,6,700,856,0.0,-5.0,545
7009726,12,13,6,1240,1437,9.0,11.0,533


In [25]:
# Percentage of missing values
data1.isnull().sum()/len(data)*100

Month         0.000000
DayofMonth    0.000000
DayOfWeek     0.000000
CRSDepTime    0.000000
CRSArrTime    0.000000
ArrDelay      2.206919
DepDelay      1.943670
Distance      0.000000
dtype: float64

In [26]:
# Remove rows missing data
data2 = data1.dropna()
# Confirm that observations missing data were removed  
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6855029 entries, 0 to 7009727
Data columns (total 8 columns):
Month         int64
DayofMonth    int64
DayOfWeek     int64
CRSDepTime    int64
CRSArrTime    int64
ArrDelay      float64
DepDelay      float64
Distance      int64
dtypes: float64(2), int64(6)
memory usage: 470.7 MB


In [27]:
# Count the number of rows for each type
data2.groupby('DepDelay').size()

DepDelay
-534.0     1
-92.0      1
-79.0      1
-71.0      1
-70.0      3
          ..
 1597.0    1
 1710.0    1
 1952.0    1
 2457.0    1
 2467.0    1
Length: 1129, dtype: int64

In [28]:
# Count the number of rows for each type
data2.groupby('ArrDelay').size()

ArrDelay
-519.0     1
-129.0     1
-109.0     1
-92.0      1
-91.0      1
          ..
 1655.0    1
 1707.0    1
 1951.0    1
 2453.0    1
 2461.0    1
Length: 1153, dtype: int64

In [29]:
data2['OutputDep']=(data2.DepDelay>30).astype("float64")

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [30]:
# Defind calculate prevalence function
def calc_prevalence(y_actual):
    return (sum(y_actual)/len(y_actual))

In [31]:
# OutputDep Prevalence
print('Prevalence:%.3f'%calc_prevalence(data2['OutputDep'].values))

Prevalence:0.118


In [32]:
data2['OutputArr']=(data2.ArrDelay>30).astype("float64")

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [34]:
# OutputArr Prevalence
print('Prevalence:%.3f'%calc_prevalence(data2['OutputArr'].values))

Prevalence:0.131


In [35]:
# Show data 2
data2

,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,ArrDelay,DepDelay,Distance,OutputDep,OutputArr
0,1,3,4,1955,2225,-14.0,8.0,810,0.0,0.0
1,1,3,4,735,1000,2.0,19.0,810,0.0,0.0
2,1,3,4,620,750,14.0,8.0,515,0.0,0.0
3,1,3,4,930,1100,-6.0,-4.0,515,0.0,0.0
4,1,3,4,1755,1925,34.0,34.0,515,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...
7009723,12,13,6,959,1150,14.0,3.0,533,0.0,0.0
7009724,12,13,6,835,1023,-2.0,-1.0,874,0.0,0.0
7009725,12,13,6,700,856,0.0,-5.0,545,0.0,0.0
7009726,12,13,6,1240,1437,9.0,11.0,533,0.0,0.0


#### One-hot-encoding Month, DayofMonth, DayOfWeek 

In [36]:
cols_cat = ['Month','DayofMonth','DayOfWeek']
cols_cat

['Month', 'DayofMonth', 'DayOfWeek']

In [37]:
# Count the number of categories for each variable
data2[cols_cat].nunique()

Month         12
DayofMonth    31
DayOfWeek      7
dtype: int64

In [38]:
# Convert type of Month, DayofMonth, DayOfWeek to object
data2[cols_cat] = data2[cols_cat].astype('object')

C:\Users\Admin\Anaconda3\lib\site-packages\pandas\core\frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [39]:
# One-hot-encoding
data_cat = pd.get_dummies(data2[cols_cat],drop_first = True)
data_cat.head(10)

,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,...,DayofMonth_28,DayofMonth_29,DayofMonth_30,DayofMonth_31,DayOfWeek_2,DayOfWeek_3,DayOfWeek_4,DayOfWeek_5,DayOfWeek_6,DayOfWeek_7
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [40]:
# Add one-hot-encoding columns to data2
data3 = pd.concat([data2,data_cat], axis = 1)

In [42]:
# Drop 'Month','DayofMonth','DayOfWeek','ArrDelay','DepDelay' columns
data3_drop= data3.drop(['Month','DayofMonth','DayOfWeek','ArrDelay','DepDelay'], axis=1)
data3_drop

,CRSDepTime,CRSArrTime,Distance,OutputDep,OutputArr,Month_2,Month_3,Month_4,Month_5,Month_6,...,DayofMonth_28,DayofMonth_29,DayofMonth_30,DayofMonth_31,DayOfWeek_2,DayOfWeek_3,DayOfWeek_4,DayOfWeek_5,DayOfWeek_6,DayOfWeek_7
0,1955,2225,810,0.0,0.0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,735,1000,810,0.0,0.0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,620,750,515,0.0,0.0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,930,1100,515,0.0,0.0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,1755,1925,515,1.0,1.0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7009723,959,1150,533,0.0,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
7009724,835,1023,874,0.0,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
7009725,700,856,545,0.0,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
7009726,1240,1437,533,0.0,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


## 2. Apply models in Naïve Bayes, Logistic Regression, DecisionTree, Random Forest, Gradient Boosting and SVM.

In [44]:
# Train_test_Split
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(data3_drop.drop(['OutputArr'], axis=1), data3_drop['OutputArr'],
                                                    test_size=0.2, random_state=42)

In [45]:
# describes info about train and test set 
print("Number transactions X_train dataset: ", train_x.shape) 
print("Number transactions y_train dataset: ", train_y.shape) 
print("Number transactions X_test dataset: ", test_x.shape) 
print("Number transactions y_test dataset: ", test_y.shape)

Number transactions X_train dataset:  (5484023, 51)
Number transactions y_train dataset:  (5484023,)
Number transactions X_test dataset:  (1371006, 51)
Number transactions y_test dataset:  (1371006,)


### Naive Bayes

In [46]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(train_x,train_y)

GaussianNB(priors=None, var_smoothing=1e-09)

In [47]:
# Make predictions using the testing set
y_pred = nb.predict(test_x)

In [48]:
from sklearn import metrics
# Print precision, recall and f1 score
print(metrics.classification_report(test_y, y_pred))

              precision    recall  f1-score   support

         0.0       0.97      0.95      0.96   1190732
         1.0       0.72      0.79      0.75    180274

   micro avg       0.93      0.93      0.93   1371006
   macro avg       0.84      0.87      0.86   1371006
weighted avg       0.94      0.93      0.93   1371006



In [49]:
print(metrics.confusion_matrix(test_y, y_pred))

[[1134103   56629]
 [  37051  143223]]


### Decision Tree

In [50]:
from sklearn import tree
decision_tree = tree.DecisionTreeClassifier()
decision_tree.fit(train_x,train_y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [51]:
# Make predictions using the testing set
y_pred_tree = decision_tree.predict(test_x)

In [52]:
# Print precision, recall and f1 score
print(metrics.classification_report(test_y, y_pred_tree))

              precision    recall  f1-score   support

         0.0       0.96      0.95      0.95   1190732
         1.0       0.69      0.70      0.70    180274

   micro avg       0.92      0.92      0.92   1371006
   macro avg       0.82      0.83      0.83   1371006
weighted avg       0.92      0.92      0.92   1371006



In [53]:
# Print confusion_matrix
print(metrics.confusion_matrix(test_y, y_pred))

[[1134103   56629]
 [  37051  143223]]


### Logistic Regression

In [54]:
from sklearn.linear_model import LogisticRegression
# all parameters not specified are set to their defaults
#logisticRegr = LogisticRegression()
logisticRegr = LogisticRegression(class_weight='balanced')
logisticRegr.fit(train_x,train_y)
y_pred_logis = logisticRegr.predict(test_x)

THRESHOLD = 0.8
y_pred_logis_threshold = np.where(logisticRegr.predict_proba(test_x)[:,1] > THRESHOLD, 1, 0)

C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [84]:
#print(metrics.classification_report(test_y, y_pred_logis))
print(metrics.classification_report(test_y, y_pred_logis_threshold))

              precision    recall  f1-score   support

         0.0       0.97      0.98      0.97   1190732
         1.0       0.87      0.78      0.82    180274

   micro avg       0.96      0.96      0.96   1371006
   macro avg       0.92      0.88      0.90   1371006
weighted avg       0.95      0.96      0.96   1371006



### Random Forest

### Gradient Boosting

### SVM

## 3. Apply PCA, SelectKBest and RFE for feature selections.

In [62]:
data3_drop_NoOutputDep = data3_drop.drop(["OutputDep","OutputArr"], axis=1)

data3_drop_NoOutputDep

,CRSDepTime,CRSArrTime,Distance,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,...,DayofMonth_28,DayofMonth_29,DayofMonth_30,DayofMonth_31,DayOfWeek_2,DayOfWeek_3,DayOfWeek_4,DayOfWeek_5,DayOfWeek_6,DayOfWeek_7
0,1955,2225,810,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,735,1000,810,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,620,750,515,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,930,1100,515,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,1755,1925,515,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7009723,959,1150,533,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
7009724,835,1023,874,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
7009725,700,856,545,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
7009726,1240,1437,533,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


#### Normalize data

In [63]:
from sklearn.preprocessing import MinMaxScaler

In [64]:
# create scaler
scaler = MinMaxScaler()

In [67]:
features = ['CRSDepTime', 'CRSArrTime', 'Distance']
x = data3_drop_NoOutputDep.loc[:, features].values
# fit and transform in one step
datanom = scaler.fit_transform(x)
datanom

C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


array([[0.82874099, 0.94319627, 0.16138154],
       [0.3115727 , 0.42390844, 0.16138154],
       [0.26282323, 0.31793133, 0.10179762],
       ...,
       [0.29673591, 0.36286562, 0.107857  ],
       [0.52564646, 0.60915642, 0.10543325],
       [0.467571  , 0.60110216, 0.17430822]])

In [69]:
# Convert Array to dataframe
feature_convert=['CRSDepTime', 'CRSArrTime', 'Distance']
rows= [x for x in range(6855029 )]

datascaled_nom=pd.DataFrame(datanom, columns=feature_convert)
datascaled_nom.index=rows
datascaled_nom.head()

,CRSDepTime,CRSArrTime,Distance
0,0.828741,0.943196,0.161382
1,0.311573,0.423908,0.161382
2,0.262823,0.317931,0.101798
3,0.394235,0.466299,0.101798
4,0.743959,0.816024,0.101798


In [71]:
# Add one-hot-encoding columns to datascaled_nom
#datascaled_nom_add = pd.concat([datascaled_nom,data_cat], axis = 1)

#MemoryError

In [76]:
from sklearn.decomposition import PCA
pca=PCA(n_components=2)
principalComponents=pca.fit_transform(datascaled_nom)
principalDf=pd.DataFrame(data=principalComponents, columns=['principal component 1', 'principal component 2'])

#,'principal component 3','principal component 4','principal component 5',
#'principal component 6','principal component 7'])

principalDf

,principal component 1,principal component 2
0,-0.408800,0.016072
1,0.323863,0.026474
2,0.434824,-0.038263
3,0.236764,-0.038808
4,-0.257646,-0.046025
...,...,...
6855024,0.212901,-0.034359
6855025,0.287447,0.034104
6855026,0.378838,-0.031612
6855027,0.042660,-0.036492


In [77]:
element_ratio = pca.explained_variance_ratio_
element_ratio

array([0.77174515, 0.14029078])

In [78]:
sum(element_ratio)

0.9120359281286085

### Apply PCA on Logistic Regression Model

In [79]:
# Train_test_Split
from sklearn.model_selection import train_test_split
train_x_PCA, test_x_PCA, train_y_PCA, test_y_PCA = train_test_split(principalDf, data3_drop['OutputArr'],
                                                    test_size=0.2, random_state=42)

In [82]:
#from sklearn.linear_model import LogisticRegression
# all parameters not specified are set to their defaults
#logisticRegr = LogisticRegression()
logisticRegr_PCA = LogisticRegression(class_weight='balanced')
logisticRegr_PCA.fit(train_x_PCA,train_y_PCA)
y_pred_logis_PCA = logisticRegr_PCA.predict(test_x_PCA)

THRESHOLD = 0.8
y_pred_logis_threshold_PCA = np.where(logisticRegr_PCA.predict_proba(test_x_PCA)[:,1] > THRESHOLD, 1, 0)

C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [83]:
#print(metrics.classification_report(test_y, y_pred_logis))
print(metrics.classification_report(test_y_PCA, y_pred_logis_threshold_PCA))

C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

         0.0       0.87      1.00      0.93   1190732
         1.0       0.00      0.00      0.00    180274

   micro avg       0.87      0.87      0.87   1371006
   macro avg       0.43      0.50      0.46   1371006
weighted avg       0.75      0.87      0.81   1371006



C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


- Precision, recall, f1-score come to zero for delay arrival prediction.
- The Logistic Regression model used PCA cannot predict accurately.

### Apply Select K Best

### Apply RFE

## 4. Using gridsearchCV to obtain best parameters for models.

## 5. Compare performances among models, write up analysis why the model is good or bad in the algorithmic approach

### Evaluate model

## 6. Conclusion